# <center style="background-color:#606F66;color: white; vertical-align: middle; padding:20px 5px;">Aerial Semantic Segmentation Drone </center>

## <center style="background-color:#EF6145;color: white; vertical-align: middle; padding:10px 5px;">Importing Libraries</center>

In [ ]:
import cv2 as cv
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import tensorflow as tf
from tensorflow import keras
import os

import imgaug as ia
from imgaug import augmenters

from Augmentaion_Pipeline import iter_path_list
from Decode_csv import decode_csv
from functools import partial

from sklearn.model_selection import train_test_split
from sklearn.utils import shuffle


# <center style="background-color:#54A57C;color: white; vertical-align: middle; padding:10px 5px;">Preprocessing</center>

## <center style="background-color:#EF6145;color: white; vertical-align: middle; padding:10px 5px;">Image Path glob</center>

In [ ]:
IMG_HEIGHT = 800
IMG_WIDTH  = 1200
IMG_CHANNELS = 3
Num_Classes = 23
reshape_dims = (IMG_HEIGHT,IMG_WIDTH)
combined=[Num_Classes,reshape_dims]# to pass as argument for map function
batch_size = 16

In [ ]:
SEG_PNG_LIST = tf.io.gfile.glob("./Data/label_images_semantic/*.png")
ORIGINAL_JPG_LIST = tf.io.gfile.glob('./Data/original_images/*.jpg')
index = [os.path.splitext(filename)[0] for filename in os.listdir('./Data/original_images/')]

SEG_PNG_LIST.sort()
ORIGINAL_JPG_LIST.sort()
index.sort()


Orginal_images_aug_path='./Data/original_images_aug'


Segmented_images_aug_path='./Data/label_images_semantic_aug'




## <center style="background-color:#EF6145;color: white; vertical-align: middle; padding:10px 5px;">Data augmentaion</center>

In [ ]:
rot_90 = augmenters.Rot90(1) #rot_90 = augmenters.Rot90(times) #time=1 =>90deg, time=2 =>180deg, time=3 =>270deg
rot_180 = augmenters.Rot90(2) 
rot_270 = augmenters.Rot90(3)
flip_hor = augmenters.Fliplr(1) 
fli_vert = augmenters.Flipud(1)
perspective_transform = augmenters.PerspectiveTransform(scale=(0.01, 0.15))

pipeline = [rot_90, rot_180, rot_270, flip_hor, fli_vert,perspective_transform]

In [ ]:
iter_path_list(index,SEG_PNG_LIST,".png",Segmented_images_aug_path,pipeline)
iter_path_list(index,ORIGINAL_JPG_LIST,".jpg",Orginal_images_aug_path,pipeline)

In [ ]:
ORIGINAL_JPG_LIST_with_aug = ORIGINAL_JPG_LIST+tf.io.gfile.glob('./Data/original_images_aug/*.jpg')
SEG_PNG_LIST_with_aug      = SEG_PNG_LIST+tf.io.gfile.glob('./Data/label_images_semantic_aug/*.png')
index_with_aug=index+[os.path.splitext(filename)[0] for filename in os.listdir('./Data/original_images_aug/')]

## <center style="background-color:#EF6145;color: white; vertical-align: middle; padding:10px 5px;">Creating Dataframe from paths</center>

In [ ]:
df=pd.DataFrame((index_with_aug,ORIGINAL_JPG_LIST_with_aug,SEG_PNG_LIST_with_aug)).T
df.rename(columns={
    0: 'index',
    1: 'Orginal',
    2: 'Segmented'},
          inplace=True)
df.set_index('index', inplace= True)
df.to_csv('DataFrame.csv')

df = shuffle(df)


In [ ]:
X_train,X_val = train_test_split(df, test_size=0.15, random_state=42)
X_val,X_test = train_test_split(X_val, test_size=0.15, random_state=42)